# dataset

In [1]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=pronet",
        "task=subgraph_distance_prediction",   
        # "task=structure_denoising",    
        "dataset=afdb_swissprot_v4",
        # "dataset.datamodule.num_workers=1",
        "features=fe_subgraph",
        "encoder.node_embedding=False",
        "encoder.pretraining=True",
        # "+aux_task=none",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/15/24 16:52:30] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=681391;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=919758;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=950150;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=354228;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=674095;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=7336;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/15/24 16:52:32] INFO     Hydra initialised at                                                    ]8;id=887169;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=689145;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

In [2]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=363250;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=869193;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=906634;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=30251;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=202108;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=431656;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=415280;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=556987;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [3]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'name', 'seed', 'num_workers', 'task_name', 'test'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$APP_CONFIG_G

In [4]:
from omegaconf import OmegaConf

In [5]:
from proteinworkshop.datasets.atom3d_datamodule import ATOM3DDataModule

In [6]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)
# print("Original config:\n", OmegaConf.to_yaml(cfg))
mutable_cfg = OmegaConf.to_container(cfg.dataset.datamodule, resolve=True)
mutable_cfg = OmegaConf.create(mutable_cfg)
# print("Cloned config:\n", OmegaConf.to_yaml(mutable_cfg))
# Instantiate the datamodule with the mutable configuration
datamodule = hydra.utils.instantiate(mutable_cfg)
datamodule.setup("fit")
dl = datamodule.train_dataloader()

for i in dl:
    print(i)
    break

[07/15/24 16:52:33] DEBUG    CUDA available: True                                                     ]8;id=985362;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=136322;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=686918;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=456560;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=3652;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=953445;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=732969;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=652712;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

100%|██████████| 542378/542378 [00:00<00:00, 4760535.83it/s]
Processing...
Done!
100%|██████████| 542378/542378 [00:00<00:00, 4793033.28it/s]
Processing...
Done!
100%|██████████| 542378/542378 [00:00<00:00, 4802301.44it/s]
Processing...
Done!
100%|██████████| 542378/542378 [00:00<00:00, 4785360.59it/s]
Processing...
Done!
/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning:

This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 28, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



ProteinBatch(fill_value=[32], atom_list=[32], coords=[8139, 37, 3], residues=[32], residue_id=[32], chains=[8139], residue_type=[8139], b_factor=[8139], id=[32], x=[8139], seq_pos=[8139, 1], batch=[8139], ptr=[33])


In [7]:
print(i.x)

tensor[8139] 32Kb all_zeros


In [8]:
from proteinworkshop.models.base import BenchMarkModel
import lightning as L

model: L.LightningModule = BenchMarkModel(cfg)

[07/15/24 16:52:38] INFO     Instantiating encoder...                                                   ]8;id=639553;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=922658;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#413\413]8;;\

[07/15/24 16:52:39] INFO     ProNet(                                                                    ]8;id=501098;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=85411;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#415\415]8;;\
                               (feature0): d_theta_phi_emb()                                                       
                               (feature1): d_angle_emb()                                                           
                               (embedding): Embedding(26, 128)                                                     
                               (interaction_blocks): ModuleList(                                                   
                                 (0-3): 4 x InteractionBlock(                                                      
                                   (dropout): Dropout(p=0.3, inplace=False)                                        
                                   (conv0): EdgeGraphConv(128, 128)                                                
                                   (conv1): EdgeGraphConv(128, 128)                                                
                                   (conv2): EdgeGraphConv(128, 128)                                                
                                   (lin_feature0): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_feature1): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_feature2): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_1): Linear()                                                               
                                   (lin_2): Linear()                                                               
                                   (lin0): Linear()                                                                
                                   (lin1): Linear()                                                                
                                   (lin2): Linear()                                                                
                                   (lins_cat): ModuleList(                                                         
                                     (0-2): 3 x Linear()                                                           
                                   )                                                                               
                                   (lins): ModuleList(                                                             
                                     (0-1): 2 x Linear()                                                           
                                   )                                                 

                    INFO     Instantiating decoders...                                                  ]8;id=782367;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=179633;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#417\417]8;;\

                    INFO     Building subgraph_distances decoder. Output dim None                       ]8;id=504821;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=330470;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#252\252]8;;\

                    INFO     {'_target_': 'proteinworkshop.models.decoders.mlp_dist.MLP_Pred_Dist',     ]8;id=950707;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=907947;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#255\255]8;;\
                             'hidden_channels': 128, 'input': 'fused_repr'}                                        

...

                    INFO     ['rmse']                                                                   ]8;id=222315;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=672162;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#434\434]8;;\

                    INFO     Instantiating featuriser...                                                ]8;id=488925;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=985596;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#436\436]8;;\

                    INFO     ProteinFeaturiser(representation=CA,                                       ]8;id=243536;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=712545;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#438\438]8;;\
                             scalar_node_features=['amino_acid_one_hot'], vector_node_features=[],                 
                             edge_types=[], scalar_edge_features=[], vector_edge_features=[])                      

                    INFO     Instantiating task transform...                                            ]8;id=78549;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=133528;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#440\440]8;;\

                    INFO     None                                                                       ]8;id=549115;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=328127;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#444\444]8;;\

In [9]:
import torch

for i in dl:
    batch = model.featurise(i)
    print(batch)
    break
batch_data = batch
# z, pos, _ = torch.squeeze(batch_data.x.long()), batch_data.pos, batch_data.batch
# print(z.shape, pos.shape) # torch.Size([5762, 16]) torch.Size([5762, 3]) torch.Size([5762])
# from proteinworkshop.models.graph_encoders.pronet import ProNet
# mutable_cfg = OmegaConf.to_containexr(cfg.encoder, resolve=True)
# mutable_cfg = OmegaConf.create(mutable_cfg)
# print("Cloned config:\n", OmegaConf.to_yaml(mutable_cfg))
# Instantiate the datamodule with the mutable configuration
# encoder = hydra.utils.instantiate(mutable_cfg)
# encoder = ProNet()
# print(model)
# print(encoder)
out = model(batch)
# out = encoder(batch)
print(out)
print(batch.device)

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning:

This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 28, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



ProteinBatch(fill_value=[32], atom_list=[32], coords=[11302, 37, 3], residues=[32], residue_id=[32], chains=[11302], residue_type=[11302], b_factor=[11302], id=[32], x=[11302, 23], seq_pos=[11302, 1], batch=[11302], ptr=[33], pos=[11302, 3], edge_index=[2, 203101], subgraphs=[1116, 137], subgraph_distances=[1116], subgraph_lengths=[1116])
{'fused_repr': tensor[1116, 256] n=285696 (1.1Mb) x∈[-0.479, 0.407] μ=0.007 σ=0.079 grad StackBackward0, 'subgraph_distances': tensor[1116, 1] 4.4Kb x∈[-0.031, 0.020] μ=-0.005 σ=0.010 grad AddmmBackward0}


ProteinBatch(fill_value=[32], atom_list=[32], coords=[10916, 37, 3], residues=[32], residue_id=[32], chains=[10916], residue_type=[10916], b_factor=[10916], id=[32], x=[10916, 23], seq_pos=[10916, 1], batch=[10916], ptr=[33], pos=[10916, 3], edge_index=[2, 153666], subgraphs=[1076, 149], dist=[1076], subgraph_lengths=[1076])
b ProteinBatch(fill_value=[32], atom_list=[32], coords=[10916, 37, 3], residues=[32], residue_id=[32], chains=[10916], residue_type=[10916], b_factor=[10916], id=[32], x=[10916, 23], seq_pos=[10916, 1], batch=[10916], ptr=[33], pos=[10916, 3], edge_index=[2, 153666], subgraphs=[1076, 149], dist=[1076], subgraph_lengths=[1076])
o {'fused_repr': tensor[1076, 256] n=275456 (1.1Mb) x∈[-0.880, 0.835] μ=0.012 σ=0.141 grad StackBackward0}
h subgraph_distance
O {'fused_repr': tensor[1076, 256] n=275456 (1.1Mb) x∈[-0.880, 0.835] μ=0.012 σ=0.141 grad StackBackward0, 'subgraph_distance': tensor[1076, 1] 4.2Kb x∈[-0.010, 0.043] μ=0.014 σ=0.014 grad AddmmBackward0}
{'fused_repr': tensor[1076, 256] n=275456 (1.1Mb) x∈[-0.880, 0.835] μ=0.012 σ=0.141 grad StackBackward0, 'subgraph_distance': tensor[1076, 1] 4.2Kb x∈[-0.010, 0.043] μ=0.014 σ=0.014 grad AddmmBackward0}

In [13]:
print(batch_data.x)
print(batch_data.edge_index)
print(batch_data.atom_list)
print(batch_data.coords)
print(batch_data.residues)
print(batch_data.id)
print(batch_data.residue_id)
print(batch_data.residue_type)
print(batch_data.chains)
print(batch_data.graph_y)
print(batch_data.amino_acid_one_hot)
print(batch_data.batch)
print(batch_data.ptr)
print(batch_data.pos)
print(batch_data.edge_index)
print(batch_data.edge_type)
print(batch_data.edge_attr)

tensor[5436, 23] n=125028 (0.5Mb) x∈[0., 1.000] μ=0.043 σ=0.204
tensor[2, 86976] i64 n=173952 (1.3Mb) x∈[0, 5435] μ=2.717e+03 σ=1.569e+03
['N', 'CA', 'C', 'O', 'CB', 'OG', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'OD1', 'ND2', 'CG1', 'CG2', 'CD', 'CE', 'NZ', 'OD2', 'OE1', 'NE2', 'OE2', 'OH', 'NE', 'NH1', 'NH2', 'OG1', 'SD', 'ND1', 'SG', 'NE1', 'CE3', 'CZ2', 'CZ3', 'CH2', 'OXT']
tensor[5436, 37, 3] n=603396 (2.3Mb) x∈[-79.883, 167.761] μ=5.206 σ=16.581
[['THR', 'LEU', 'THR', 'ILE', 'ASP', 'ASP', 'GLY', 'ASN', 'ILE', 'GLU', 'ILE', 'VAL', 'GLY', 'THR', 'GLY', 'VAL', 'LYS', 'GLY', 'LYS', 'LEU', 'PRO', 'THR', 'VAL', 'TRP', 'LEU', 'GLN', 'TYR', 'GLY', 'GLN', 'VAL', 'ASN', 'LEU', 'LYS', 'ALA', 'SER', 'GLY', 'GLY', 'ASN', 'GLY', 'LYS', 'TYR', 'THR', 'TRP', 'ARG', 'SER', 'ALA', 'ASN', 'PRO', 'ALA', 'ILE', 'ALA', 'SER', 'VAL', 'ASP', 'ALA', 'SER', 'SER', 'GLY', 'GLN', 'VAL', 'THR', 'LEU', 'LYS', 'GLU', 'LYS', 'GLY', 'THR', 'THR', 'THR', 'ILE', 'SER', 'VAL', 'ILE', 'SER', 'SER', 'ASP', 'ASN', 'GLN

In [10]:
import torch

# Assuming `dl` is your data loader and `model.featurise(i)` returns a batch with `.x`, `.pos`, and `.batch` attributes
unique_values = set()

for i in dl:
    batch = model.featurise(i)
    z, pos, _ = torch.squeeze(batch.x.long()), batch.pos, batch.batch
    
    # Reshape z to a flat tensor to extract unique values
    z_flat = z.view(-1)
    unique_values.update(z_flat.unique().tolist())

print(unique_values)


KeyboardInterrupt: 

afdb_swissprot_v4

ProteinBatch(fill_value=[32], atom_list=[32], coords=[12984, 37, 3], residues=[32], residue_id=[32], chains=[12984], residue_type=[12984], b_factor=[12984], id=[32], x=[12984], seq_pos=[12984, 1], batch=[12984], ptr=[33])



fold_fold

DataProteinBatch(fill_value=1e-05, atom_list=[37], coords=[6159, 37, 3], residues=[32], id=[32], residue_id=[32], residue_type=[6159], chains=[6159], graph_y=[32], x=[6159], amino_acid_one_hot=[6159, 23], seq_pos=[6159, 1], batch=[6159], ptr=[33])

No feature:

DataProteinBatch(fill_value=1e-05, atom_list=[37], coords=[4221, 37, 3], residues=[32], id=[32], residue_id=[32], residue_type=[4221], chains=[4221], graph_y=[32], x=[4221, 16], amino_acid_one_hot=[4221, 23], seq_pos=[4221, 1], batch=[4221], ptr=[33], pos=[4221, 3], edge_index=[2, 67536], edge_type=[1, 67536], num_relation=1, edge_attr=[67536, 1])

ca_base:

DataProteinBatch(fill_value=1e-05, atom_list=[37], coords=[5388, 37, 3], residues=[32], id=[32], residue_id=[32], residue_type=[5388], chains=[5388], graph_y=[32], x=[5388, 23], amino_acid_one_hot=[5388, 23], seq_pos=[5388, 1], batch=[5388], ptr=[33], pos=[5388, 3], edge_index=[2, 86208], edge_type=[1, 86208], num_relation=1, edge_attr=[86208, 1])
torch.Size([5388, 23]) torch.Size([5388, 3])

# train

In [2]:
import copy
import sys
from typing import List, Optional

import graphein
import hydra
import lightning as L
import lovely_tensors as lt
import torch
import torch.nn as nn
import torch_geometric
from graphein.protein.tensor.dataloader import ProteinDataLoader
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.loggers import Logger
from loguru import logger as log
from omegaconf import DictConfig

from proteinworkshop import (
    constants,
    register_custom_omegaconf_resolvers,
    utils,
)
from proteinworkshop.configs import config
from proteinworkshop.models.base import BenchMarkModel

In [6]:
import pytorch_lightning as pl

In [3]:
from proteinworkshop.models.base import BenchMarkModel
from proteinworkshop.models.graph_encoders.pronet import ProNet
model: L.LightningModule = BenchMarkModel(cfg)
datamodule: L.LightningDataModule = hydra.utils.instantiate(
        cfg.dataset.datamodule
    )
callbacks: List[Callback] = utils.callbacks.instantiate_callbacks(
        cfg.get("callbacks")
    )
logger: List[Logger] = utils.loggers.instantiate_loggers(cfg.get("logger"))
trainer: L.Trainer = hydra.utils.instantiate(
        cfg.trainer, callbacks=callbacks, logger=logger
    )
# trainer.connectors.logger_connector.result.ResultCollection.extract_batch_size = 32
with torch.no_grad():
    datamodule.setup(stage="lazy_init")  # type: ignore
    batch = next(iter(datamodule.val_dataloader()))
    log.info(f"Unfeaturized batch: {batch}")
    batch = model.featurise(batch)
    log.info(f"Featurized batch: {batch}")
    log.info(f"Example labels: {model.get_labels(batch)}")
    # Check batch has required attributes
    for attr in model.encoder.required_batch_attributes:  # type: ignore
        if not hasattr(batch, attr):
            raise AttributeError(
                f"Batch {batch} does not have required attribute: {attr} ({model.encoder.required_batch_attributes})"
            )
    out = model(batch)
    log.info(f"Model output: {out}")
    del batch, out


[07/08/24 00:00:30] INFO     Instantiating encoder...                                                   ]8;id=989000;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=154936;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#411\411]8;;\

[07/08/24 00:00:31] INFO     ProNet(                                                                    ]8;id=736321;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=333773;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#413\413]8;;\
                               (feature0): d_theta_phi_emb()                                                       
                               (feature1): d_angle_emb()                                                           
                               (embedding): Embedding(26, 128)                                                     
                               (interaction_blocks): ModuleList(                                                   
                                 (0-3): 4 x InteractionBlock(                                                      
                                   (dropout): Dropout(p=0, inplace=False)                                          
                                   (conv0): EdgeGraphConv(128, 128)                                                
                                   (conv1): EdgeGraphConv(128, 128)                                                
                                   (conv2): EdgeGraphConv(128, 128)                                                
                                   (lin_feature0): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_feature1): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_feature2): TwoLinear(                                                      
                                     (lin1): Linear()                                                              
                                     (lin2): Linear()                                                              
                                   )                                                                               
                                   (lin_1): Linear()                                                               
                                   (lin_2): Linear()                                                               
                                   (lin0): Linear()                                                                
                                   (lin1): Linear()                                                                
                                   (lin2): Linear()                                                                
                                   (lins_cat): ModuleList(                                                         
                                     (0-2): 3 x Linear()                                                           
                                   )                                                                               
                                   (lins): ModuleList(                                                             
                                     (0-1): 2 x Linear()                                                           
                                   )                                                

                    INFO     Instantiating decoders...                                                  ]8;id=24896;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=631267;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#415\415]8;;\

                    INFO     Building graph_label decoder. Output dim 1195                              ]8;id=853950;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=610684;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#250\250]8;;\

                    INFO     {'_target_': 'proteinworkshop.models.decoders.mlp_decoder.MLPDecoder',     ]8;id=583621;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=379383;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#253\253]8;;\
                             'hidden_dim': [512, 512, 512], 'dropout': 0.0, 'activations': ['relu',                
                             'relu', 'relu', 'none'], 'skip': 'concat', 'out_dim':                                 
                             '${dataset.num_classes}', 'input': 'graph_embedding'}                                 

...

100%|██████████| 736/736 [00:00<00:00, 4782.98it/s]


[07/08/24 00:00:50] INFO     Unfeaturized batch: DataProteinBatch(fill_value=1e-05, atom_list=[37], ]8;id=384509;file:///tmp/ipykernel_46521/719150000.py\719150000.py]8;;\:]8;id=978798;file:///tmp/ipykernel_46521/719150000.py#18\18]8;;\
                             coords=[7747, 37, 3], residues=[32], id=[32], residue_id=[32],                        
                             residue_type=[7747], chains=[7747], graph_y=[32], x=[7747],                           
                             amino_acid_one_hot=[7747, 23], seq_pos=[7747, 1], batch=[7747],                       
                             ptr=[33])                                                                             

                    INFO     Featurized batch: DataProteinBatch(fill_value=1e-05, atom_list=[37],   ]8;id=802365;file:///tmp/ipykernel_46521/719150000.py\719150000.py]8;;\:]8;id=127592;file:///tmp/ipykernel_46521/719150000.py#20\20]8;;\
                             coords=[7747, 37, 3], residues=[32], id=[32], residue_id=[32],                        
                             residue_type=[7747], chains=[7747], graph_y=[32], x=[7747, 23],                       
                             amino_acid_one_hot=[7747, 23], seq_pos=[7747, 1], batch=[7747],                       
                             ptr=[33], pos=[7747, 3], edge_index=[2, 123952], edge_type=[1,                        
                             123952], num_relation=1, edge_attr=[123952, 1])                                       

                    INFO     Example labels: {'graph_label': tensor[32] i64 x∈[0, 128] μ=103.312    ]8;id=726125;file:///tmp/ipykernel_46521/719150000.py\719150000.py]8;;\:]8;id=949157;file:///tmp/ipykernel_46521/719150000.py#21\21]8;;\
                             σ=34.218}                                                                             

                    INFO     Model output: {'graph_embedding': tensor[32, 1] x∈[-0.012, 0.031]      ]8;id=826410;file:///tmp/ipykernel_46521/719150000.py\719150000.py]8;;\:]8;id=470688;file:///tmp/ipykernel_46521/719150000.py#29\29]8;;\
                             μ=0.005 σ=0.009, 'graph_label': tensor[32, 1195] n=38240 (0.1Mb)                      
                             x∈[-0.481, 0.535] μ=0.001 σ=0.163}                                                    

In [4]:
# dl = datamodule.train_dataloader()
# for i in dl:
#     batch = model.featurise(i)
    

In [4]:
if cfg.get("compile"):
    log.info("Compiling model!")
    model = torch_geometric.compile(model, dynamic=True)

: 

In [5]:

trainer.fit(
    model=model, datamodule=datamodule, ckpt_path=cfg.get("ckpt_path")
)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


                    INFO     Downloading Protein Function. Fraction 1.0                   ]8;id=368720;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py\fold_classification.py]8;;\:]8;id=380541;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py#86\86]8;;\

                    INFO     Found SCOP structure tarfile in:                            ]8;id=759145;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py\fold_classification.py]8;;\:]8;id=104485;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py#138\138]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/dat                           
                             a/FoldClassification/pdbstyle-sel-gs-bib-95-1.75.tgz                                  

                    INFO     Found SCOPe structures in:                                  ]8;id=480277;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py\fold_classification.py]8;;\:]8;id=257298;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/fold_classification.py#151\151]8;;\

                    INFO     Preprocessing training data                                                 ]8;id=436090;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=579685;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py#87\87]8;;\

...

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:368: UserWarning:

You have overridden `on_after_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.



┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃     ┃ Name                                           ┃ Type        ┃ Params ┃ Mode  ┃    In sizes ┃   Out sizes ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 0   │ encoder                                        │ ProNet      │  1.4 M │ train │           ? │           ? │
│ 1   │ encoder.feature0                               │ d_theta_ph… │      0 │ train │  [[179136], │    [179136, │
│     │                                                │             │        │       │   [179136], │         24] │
│     │                                                │             │        │       │   [179136]] │             │
│ 2   │ encoder.feature1                               │ d_angle_emb │      0 │ train │  [[179136], │    [179136, │
│     │                                                │             │        │       │   [179136]] │         12] │
│ 3   │ encoder.embedding                              │ Embedding   │  3.3 K │ train │     [11196] │     [11196, │
│     │                                                │             │        │       │             │        128] │
│ 4   │ encoder.interaction_blocks                     │ ModuleList  │  1.4 M │ train │           ? │           ? │
│ 5   │ encoder.interaction_blocks.0                   │ Interactio… │  340 K │ train │    [[11196, │     [11196, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │    [179136, │             │
│     │                                                │             │        │       │        24], │             │
│     │                                                │             │        │       │    [179136, │             │
│     │                                                │             │        │       │        12], │             │
│     │                                                │             │        │       │    [179136, │             │
│     │                                                │             │        │       │    16], [2, │             │
│     │                                                │             │        │       │    179136], │             │
│     │                                                │             │        │       │    [11196]] │             │
│ 6   │ encoder.interaction_blocks.0.dropout           │ Dropout     │      0 │ train │     [11196, │     [11196, │
│     │                                                │             │        │       │        128] │        128] │
│ 7   │ encoder.interaction_blocks.0.conv0             │ EdgeGraphC… │ 32.9 K │ train │    [[11196, │     [11196, │
│     │                                                │             │        │       │   128], [2, │        128] │
│     │                                                │             │        │       │    179136], │             │
│     │                                                │             │        │       │    [179136, │             │
│     │                                                │             │        │       │       128]] │             │
│ 8   │ encoder.interaction_blocks.0.conv0.aggr_module │ SumAggrega… │      0 │ train │   [[179136, │     [11196, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │   [179136], │             │
│     │                                                │             │        │       │   '?', '?', │             │
│     │                                                │             │        │       │        '?'] │             │
│ 9   │ encoder.interaction_blocks.0.conv0.lin_l       │

Trainable params: 4.0 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.0 M                                                                                                
Total estimated model params size (MB): 16

Output()

Metric val/graph_label/accuracy improved. New best score: 0.037
Metric train/loss/total improved. New best score: 6.056
Epoch 0, global step 385: 'val/graph_label/accuracy' reached 0.03668 (best 0.03668), saving model to '/home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_000-v5.ckpt' as top 1
